Copyright (c) Alliance for Sustainable Energy, LLC and Skye Analytics, Inc. See also https://github.com/NREL/ATB-calc/blob/main/LICENSE

# Process an ATB electricity technology
This Jupyter notebook is an example of how to run the ATB data processing pipeline and extract data from the ATB electricity data workbook for an individual technology. The path to the workbook on your computer needs to be updated as indicated below. The notebook will otherwise run as-is. There is commented-out code in several locations showing alternative ways to run the pipeline. 

In [32]:
import os
import sys
import pandas as pd
from datetime import datetime as dt
import plotly.express as px

sys.path.insert(0, os.path.dirname(os.getcwd()))

# Electricity technology processors
from RES.lcoe_calculator.tech_processors import (
    FixedOffShoreWindProc, FloatingOffShoreWindProc, LandBasedWindProc, DistributedWindProc,
    UtilityPvProc, CommPvProc, ResPvProc, UtilityPvPlusBatteryProc,
    CspProc, GeothermalProc, HydropowerProc, PumpedStorageHydroProc,
    PumpedStorageHydroOneResProc,
    CoalProc, NaturalGasProc, NuclearProc, BiopowerProc,
    UtilityBatteryProc, CommBatteryProc, ResBatteryProc,
    CoalRetrofitProc, NaturalGasRetrofitProc, NaturalGasFuelCellProc
)

from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

## Load the ATB electricity workbook
Update the path for `atb_electricity_workbook` to point to your copy of the electricity workbook downloaded from the [ATB website](https://atb.nrel.gov).


In [ ]:
# The below line MUST be updated to reflect the location of the ATB workbook on your computer
atb_electricity_workbook = '../../data/downloaded_data/NREL/ATB/2024_v3_Workbook.xlsx'
# from openpyxl import load_workbook 
# wb = load_workbook(atb_electricity_workbook, data_only=True)

## Select CRP and financial case and technology

CRP - Cost Recovery period

In [ ]:
# The CRP may be one of 20, 30, or 'TechLife' in quotes
crp = 20
# crp = 30
# crp = 'TechLife'

# The financial case can be either 'R&D' or 'Market'
# financial_case = 'R&D'
financial_case = 'Market'

# Finally, choose a technology processor from the list imported from lcoe_calculator.tech_processors 
 
proc_wind = LandBasedWindProc(atb_electricity_workbook, crp=crp, case=financial_case)

# proc = NuclearProc(atb_electricity_workbook, crp=crp, case=financial_case)
proc_solar = UtilityPvProc(atb_electricity_workbook, crp=crp, case=financial_case)
# proc = NaturalGasProc(atb_electricity_workbook, crp=crp, case=financial_case)
# proc = NuclearProc(atb_electricity_workbook, crp=crp, case=financial_case)

### View techonology meta-data

In [ ]:
proc_solar.get_meta_data()

In [ ]:
proc_wind.get_meta_data()

## Run the processor and test the calculated LCOE and CAPEX against the values in the workbook. 
Note that not all technologies have LCOE and CAPEX. 

In [ ]:
# Run the pipeline
proc_solar.run()

In [ ]:
# Test LCOE and CAPEX
proc_solar.test_capex()
# proc_solar.test_lcoe()

In [ ]:
proc_wind.run()

In [ ]:
# Test LCOE and CAPEX
proc_wind.test_capex()
# proc_wind.test_lcoe()

## View results
Various metrics are available on the technology processors once they have been `run()`. Note that not all technologies have all metrics. Several typical metrics are shown below. The full list of available metrics can be determined by viewing the code in `lcoe_calculator/base_processor.py` and the technology of interest in `lcoe_calculator/tech_processors.py`. Note that all metrics are stored in pandas data frames, as indicated by the `df_` prefix. Any metric may be copied to the clipboard (e.g., for viewing in a spreadsheet) with the `.to_clipboar()` method. For example: `proc.df_lcoe.to_clipboard()`. 

In [ ]:
wind_tech='Land-Based Wind - Class 9 - Technology 3/Moderate'

In [47]:
# proc_wind.df_capex[proc_wind.df_capex.index.str.contains(wind_tech)]
# proc_wind.df_lcoe[proc_wind.df_lcoe.index.str.contains(wind_tech)]

In [45]:
# Plot LCOE by year for Offshore Wind - Class 1/Advanced using Plotly
fig = px.line(proc_wind.df_capex.loc[wind_tech], 
              labels={"index": "Year", "value": "Capex ($/W)"},
              title=f"Capex for {wind_tech}")
fig.update_layout(xaxis_title="Year", yaxis_title="capex ($/kW)")
fig.show()

In [48]:
# Plot LCOE by year for Offshore Wind - Class 1/Advanced using Plotly
fig = px.line(proc_wind.df_ncf.loc[wind_tech], 
              labels={"index": "Year", "value": "net CF"},
              title=f"Net CF for {wind_tech}")
fig.update_layout(xaxis_title="Year", yaxis_title="net CF")
fig.show()

In [44]:
# Plot LCOE by year for Offshore Wind - Class 1/Advanced using Plotly
fig = px.line(proc_wind.df_lcoe.loc[wind_tech], 
              labels={"index": "Year", "value": "LCOE ($/MWh)"},
              title=f"LCOE for {wind_tech}")
fig.update_layout(xaxis_title="Year", yaxis_title="LCOE ($/MWh)")
fig.show()

In [ ]:
# proc.df_lcoe

In [ ]:
import plotly.express as px

# Plot LCOE by year for Offshore Wind - Class 1/Advanced using Plotly
fig = px.line(proc_wind.df_capex.loc['Land-Based Wind - Class 3 - Technology 1/Conservative'], 
              labels={"index": "Year", "value": "Capex ($/MW)"},
              title="Capex by Year for Land-Based Wind - Class 3 - Technology 1/Conservative")
fig.update_layout(xaxis_title="Year", yaxis_title="Capex ($/MW)")
fig.show()